# Experiment 01: Airline dataset (GPU version)

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

For this experiment we used an [Azure NV24 VM](https://azure.microsoft.com/en-gb/blog/azure-n-series-general-availability-on-december-1/), which has NVIDIA M60 GPUs.

In [2]:
import os,sys
import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score)
from libs.loaders import load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.timer import Timer
from libs.utils import get_number_processors
import pkg_resources
import json
import matplotlib.pylab as plt

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

%matplotlib inline


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [3]:
%%time
df_plane = load_airline()
print(df_plane.shape)

INFO:libs.loaders:MOUNT_POINT not found in environment. Defaulting to /fileshare


(115069017, 14)
CPU times: user 1min 41s, sys: 14.4 s, total: 1min 56s
Wall time: 4min 16s


In [4]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


The first step is to convert the categorical features to numeric features.

In [5]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 54s, sys: 7.59 s, total: 2min 1s
Wall time: 1min 59s


In [6]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [7]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 1min 2s, sys: 7.1 s, total: 1min 10s
Wall time: 1min 8s


In [8]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [9]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 476 ms, sys: 408 ms, total: 884 ms
Wall time: 882 ms


In [10]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [11]:
#small dataset
df_plane_numeric_small = df_plane_numeric.sample(n=1e6).reset_index(drop=True)

/anaconda/envs/strata/lib/python3.5/site-packages/pandas/core/generic.py:2644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


Once the features are prepared, let's split the dataset into train, validation and test set.

In [12]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [13]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric_small, val_size=0, test_size=0.2)
#train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(800000, 15)
(0, 15)
(200000, 15)
CPU times: user 228 ms, sys: 0 ns, total: 228 ms
Wall time: 228 ms


In [14]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [15]:
%%time
X_train, y_train = generate_feables(train)
X_test, y_test = generate_feables(test)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.3 ms


In [16]:
%%time
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

CPU times: user 464 ms, sys: 4 ms, total: 468 ms
Wall time: 468 ms


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [17]:
results_dict = dict()

Let's start with the XGBoost model.

In [18]:
params = {'max_depth':8, 'num_round':200, 'min_child_weight':30, 'eta':0.1, 'colsample_bytree':0.80,
          'scale_pos_weight':2, 'gamma':0.1, 'reg_lamda':1, 'subsample':1,'tree_method':'exact', 'updater':'grow_gpu'}


In [19]:
with Timer() as t:
    xgb_clf_pipeline = xgb.train(params, dtrain)

In [20]:
results_dict['xgb']={
    'train_time': t.interval
}

Training XGBoost model with leave-wise growth

In [21]:
params = {'max_depth':0, 'num_round':200, 'min_child_weight':30, 'eta':0.1, 'colsample_bytree':0.80,
          'scale_pos_weight':2, 'gamma':0.1, 'reg_lamda':1, 'subsample':1,'tree_method':'hist', 
          'max_leaves':255, 'grow_policy':'lossguide', 'updater':'grow_gpu_hist'}


In [22]:
with Timer() as t:
    xgb_hist_clf_pipeline = xgb.train(params, dtrain)

In [23]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

Training LightGBM model

In [24]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=200,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  colsample_bytree=0.80,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  device='gpu',
                                  seed=77)

In [25]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

In [26]:
results_dict['lgbm']={
    'train_time': t.interval
}

As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [27]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(dtest), 0.0001, 0.9999)

In [28]:
results_dict['xgb']['test_time'] = t.interval

In [29]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(dtest), 0.0001, 0.9999)

In [30]:
results_dict['xgb_hist']['test_time'] = t.interval

In [31]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [32]:
results_dict['lgbm']['test_time'] = t.interval


### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [33]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1}
    return report

In [34]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    report = {'AUC':m_auc}
    return report

In [35]:
def binarize_prediction(y, threshold=0.5):
    y_pred = np.where(y > threshold, 1, 0)
    return y_pred

In [36]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)


In [37]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

In [38]:
results_dict['xgb']['performance'] = report_xgb

In [39]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [40]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [41]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [42]:
results_dict['lgbm']['performance'] = report_lgbm

In [43]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "AUC": 0.8310265082301329,
            "Accuracy": 0.755645,
            "F1": 0.7229393789932592,
            "Precision": 0.7851829959115314,
            "Recall": 0.6698393688213726
        },
        "test_time": 0.3814003080187831,
        "train_time": 19.809737928007962
    },
    "xgb": {
        "performance": {
            "AUC": 0.7445518608274961,
            "Accuracy": 0.56523,
            "F1": 0.6732894984031561,
            "Precision": 0.5240783093417875,
            "Recall": 0.9412839988653913
        },
        "test_time": 0.00908168000751175,
        "train_time": 6.795339136995608
    },
    "xgb_hist": {
        "performance": {
            "AUC": 0.7706911769723137,
            "Accuracy": 0.587885,
            "F1": 0.6824327187963551,
            "Precision": 0.5388298713783327,
            "Recall": 0.9303896540493976
        },
        "test_time": 0.06132808199618012,
        "train_time": 1.50854190500

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.